In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.model_selection import cross_val_score, RandomizedSearchCV, KFold, StratifiedKFold
from scipy.stats import uniform, randint
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import shap

import featuretools as ft
from sklearn.neural_network import MLPClassifier
import os
import sys

ROOT = os.path.abspath(os.path.join(os.pardir))
if ROOT not in sys.path:
    sys.path.append(ROOT)

from src.config import config
from src.xgboost_pipeline import XGBoostPipeline

pd.set_option('display.max_columns', None)

# update jupyter kernel automatically
%load_ext autoreload
%autoreload 2


In [2]:
df_raw = pd.read_excel("/Users/kewenyang/Documents/GitHub/Maybank_Classification/data/Assessment.xlsx",
                       engine='openpyxl',
                       sheet_name=1)
df_raw.head()

,C_ID,C_AGE,C_EDU,C_HSE,PC,INCM_TYP,gn_occ,NUM_PRD,CASATD_CNT,MTHCASA,MAXCASA,MINCASA,DRvCR,MTHTD,MAXTD,Asset value,HL_tag,AL_tag,pur_price_avg,UT_AVE,MAXUT,N_FUNDS,CC_AVE,MAX_MTH_TRN_AMT,MIN_MTH_TRN_AMT,AVG_TRN_AMT,ANN_TRN_AMT,ANN_N_TRX,CC_LMT,C_seg
0,1443,65,Masters,EXECUTIVE CONDOMINIUM,19250.0,6.0,PMEB,3,8.0,6896.91,4899.08,910.88,1.020768e+06,105000.00,25000.00,111896.91,NaN,NaN,NaN,NaN,NaN,NaN,13.233333,NaN,NaN,NaN,NaN,NaN,34500.0,AFFLUENT
1,1559,86,O-Levels,PRIVATE CONDOMINIUM,99018.0,2.0,PMEB,4,13.0,51714.78,35740.55,1318.25,8.326420e+00,575571.97,135026.15,627286.75,NaN,NaN,NaN,NaN,NaN,NaN,727.629167,8530.88,273.44,2296.713333,27560.56,88.0,4000.0,AFFLUENT
2,1913,69,A-Levels,NaN,10155.0,3.0,PMEB,4,1.0,5420.09,5420.09,5420.09,4.106600e-01,NaN,NaN,64161.74,NaN,NaN,NaN,58424.550000,59600.88,1.0,367.389167,523.35,122.13,283.580833,3402.97,78.0,5000.0,AFFLUENT
3,2032,70,NaN,NaN,21863.0,2.0,RETIREE,3,1.0,13596.65,13596.65,13596.65,5.397180e+03,NaN,NaN,39792.30,NaN,NaN,NaN,28863.181667,33902.58,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,AFFLUENT
4,2048,72,O-Levels,EXECUTIVE CONDOMINIUM,10641.0,NaN,BLUE COLLAR,4,2.0,51639.18,32689.93,18949.25,3.607580e+03,NaN,NaN,67258.47,NaN,NaN,NaN,15060.339167,15619.29,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,AFFLUENT


In [3]:
pipe = XGBoostPipeline()
df_raw = pipe.preprocess(df_raw, True)
df_raw.head()

Only one dataframe in entityset, changing max_depth to 1 since deeper features cannot be created


Built 342 features
Elapsed: 00:00 | Progress: 100%|██████████

Only one dataframe in entityset, changing max_depth to 1 since deeper features cannot be created



Built 18 features
Elapsed: 00:00 | Progress: 100%|██████████


,C_ID,C_EDU,C_HSE,INCM_TYP,gn_occ,C_seg,NUM_PRD,CASATD_CNT,ANN_N_TRX,MTHCASA,MAXCASA,MINCASA,CC_AVE,MAX_MTH_TRN_AMT,MIN_MTH_TRN_AMT,MTHTD,MAXTD,Asset value,AVG_TRN_AMT,ANN_TRN_AMT,CC_LMT,C_AGE,DRvCR,ANN_TRN_AMT + AVG_TRN_AMT,ANN_TRN_AMT + Asset value,ANN_TRN_AMT + CC_AVE,ANN_TRN_AMT + CC_LMT,ANN_TRN_AMT + MAXCASA,ANN_TRN_AMT + MAXTD,ANN_TRN_AMT + MAX_MTH_TRN_AMT,ANN_TRN_AMT + MINCASA,ANN_TRN_AMT + MIN_MTH_TRN_AMT,ANN_TRN_AMT + MTHCASA,ANN_TRN_AMT + MTHTD,AVG_TRN_AMT + CC_AVE,AVG_TRN_AMT + CC_LMT,AVG_TRN_AMT + MAXCASA,AVG_TRN_AMT + MAXTD,AVG_TRN_AMT + MAX_MTH_TRN_AMT,AVG_TRN_AMT + MINCASA,AVG_TRN_AMT + MIN_MTH_TRN_AMT,AVG_TRN_AMT + MTHCASA,AVG_TRN_AMT + MTHTD,Asset value + AVG_TRN_AMT,Asset value + CC_AVE,Asset value + CC_LMT,Asset value + MAXCASA,Asset value + MAXTD,Asset value + MAX_MTH_TRN_AMT,Asset value + MINCASA,Asset value + MIN_MTH_TRN_AMT,Asset value + MTHCASA,Asset value + MTHTD,CC_AVE + CC_LMT,CC_AVE + MAXCASA,CC_AVE + MAXTD,CC_AVE + MAX_MTH_TRN_AMT,CC_AVE + MINCASA,CC_AVE + MIN_MTH_TRN_AMT,CC_AVE + MTHCASA,CC_AVE + MTHTD,CC_LMT + MAXCASA,CC_LMT + MAXTD,CC_LMT + MAX_MTH_TRN_AMT,CC_LMT + MINCASA,CC_LMT + MIN_MTH_TRN_AMT,CC_LMT + MTHCASA,CC_LMT + MTHTD,MAXCASA + MAXTD,MAXCASA + MINCASA,MAXCASA + MIN_MTH_TRN_AMT,MAXCASA + MTHCASA,MAXCASA + MTHTD,MAXTD + MINCASA,MAXTD + MIN_MTH_TRN_AMT,MAXTD + MTHCASA,MAXTD + MTHTD,MAX_MTH_TRN_AMT + MAXCASA,MAX_MTH_TRN_AMT + MAXTD,MAX_MTH_TRN_AMT + MINCASA,MAX_MTH_TRN_AMT + MIN_MTH_TRN_AMT,MAX_MTH_TRN_AMT + MTHCASA,MAX_MTH_TRN_AMT + MTHTD,MINCASA + MTHCASA,MINCASA + MTHTD,MIN_MTH_TRN_AMT + MINCASA,MIN_MTH_TRN_AMT + MTHCASA,MIN_MTH_TRN_AMT + MTHTD,MTHCASA + MTHTD,ANN_TRN_AMT / AVG_TRN_AMT,ANN_TRN_AMT / Asset value,ANN_TRN_AMT / CC_AVE,ANN_TRN_AMT / CC_LMT,ANN_TRN_AMT / MAXCASA,ANN_TRN_AMT / MAXTD,ANN_TRN_AMT / MAX_MTH_TRN_AMT,ANN_TRN_AMT / MINCASA,ANN_TRN_AMT / MIN_MTH_TRN_AMT,ANN_TRN_AMT / MTHCASA,ANN_TRN_AMT / MTHTD,AVG_TRN_AMT / ANN_TRN_AMT,AVG_TRN_AMT / Asset value,AVG_TRN_AMT / CC_AVE,AVG_TRN_AMT / CC_LMT,AVG_TRN_AMT / MAXCASA,AVG_TRN_AMT / MAXTD,AVG_TRN_AMT / MAX_MTH_TRN_AMT,AVG_TRN_AMT / MINCASA,AVG_TRN_AMT / MIN_MTH_TRN_AMT,AVG_TRN_AMT / MTHCASA,AVG_TRN_AMT / MTHTD,Asset value / ANN_TRN_AMT,Asset value / AVG_TRN_AMT,Asset value / CC_AVE,Asset value / CC_LMT,Asset value / MAXCASA,Asset value / MAXTD,Asset value / MAX_MTH_TRN_AMT,Asset value / MINCASA,Asset value / MIN_MTH_TRN_AMT,Asset value / MTHCASA,Asset value / MTHTD,CC_AVE / ANN_TRN_AMT,CC_AVE / AVG_TRN_AMT,CC_AVE / Asset value,CC_AVE / CC_LMT,CC_AVE / MAXCASA,CC_AVE / MAXTD,CC_AVE / MAX_MTH_TRN_AMT,CC_AVE / MINCASA,CC_AVE / MIN_MTH_TRN_AMT,CC_AVE / MTHCASA,CC_AVE / MTHTD,CC_LMT / ANN_TRN_AMT,CC_LMT / AVG_TRN_AMT,CC_LMT / Asset value,CC_LMT / CC_AVE,CC_LMT / MAXCASA,CC_LMT / MAXTD,CC_LMT / MAX_MTH_TRN_AMT,CC_LMT / MINCASA,CC_LMT / MIN_MTH_TRN_AMT,CC_LMT / MTHCASA,CC_LMT / MTHTD,MAXCASA / ANN_TRN_AMT,MAXCASA / AVG_TRN_AMT,MAXCASA / Asset value,MAXCASA / CC_AVE,MAXCASA / CC_LMT,MAXCASA / MAXTD,MAXCASA / MAX_MTH_TRN_AMT,MAXCASA / MINCASA,MAXCASA / MIN_MTH_TRN_AMT,MAXCASA / MTHCASA,MAXCASA / MTHTD,MAXTD / ANN_TRN_AMT,MAXTD / AVG_TRN_AMT,MAXTD / Asset value,MAXTD / CC_AVE,MAXTD / CC_LMT,MAXTD / MAXCASA,MAXTD / MAX_MTH_TRN_AMT,MAXTD / MINCASA,MAXTD / MIN_MTH_TRN_AMT,MAXTD / MTHCASA,MAXTD / MTHTD,MAX_MTH_TRN_AMT / ANN_TRN_AMT,MAX_MTH_TRN_AMT / AVG_TRN_AMT,MAX_MTH_TRN_AMT / Asset value,MAX_MTH_TRN_AMT / CC_AVE,MAX_MTH_TRN_AMT / CC_LMT,MAX_MTH_TRN_AMT / MAXCASA,MAX_MTH_TRN_AMT / MAXTD,MAX_MTH_TRN_AMT / MINCASA,MAX_MTH_TRN_AMT / MIN_MTH_TRN_AMT,MAX_MTH_TRN_AMT / MTHCASA,MAX_MTH_TRN_AMT / MTHTD,MINCASA / ANN_TRN_AMT,MINCASA / AVG_TRN_AMT,MINCASA / Asset value,MINCASA / CC_AVE,MINCASA / CC_LMT,MINCASA / MAXCASA,MINCASA / MAXTD,MINCASA / MAX_MTH_TRN_AMT,MINCASA / MIN_MTH_TRN_AMT,MINCASA / MTHCASA,MINCASA / MTHTD,MIN_MTH_TRN_AMT / ANN_TRN_AMT,MIN_MTH_TRN_AMT / AVG_TRN_AMT,MIN_MTH_TRN_AMT / Asset value,MIN_MTH_TRN_AMT / CC_AVE,MIN_MTH_TRN_AMT / CC_LMT,MIN_MTH_TRN_AMT / MAXCASA,MIN_MTH_TRN_AMT / MAXTD,MIN_MTH_TRN_AMT / MAX_MTH_TRN_AMT,MIN_MTH

In [4]:
pd.set_option('display.max_rows', None)

df_raw.isnull().sum(axis=0) / df_raw.shape[0]

C_ID                                 0.0
C_EDU                                0.0
C_HSE                                0.0
INCM_TYP                             0.0
gn_occ                               0.0
C_seg                                0.0
NUM_PRD                              0.0
CASATD_CNT                           0.0
ANN_N_TRX                            0.0
MTHCASA                              0.0
MAXCASA                              0.0
MINCASA                              0.0
CC_AVE                               0.0
MAX_MTH_TRN_AMT                      0.0
MIN_MTH_TRN_AMT                      0.0
MTHTD                                0.0
MAXTD                                0.0
Asset value                          0.0
AVG_TRN_AMT                          0.0
ANN_TRN_AMT                          0.0
CC_LMT                               0.0
C_AGE                                0.0
DRvCR                                0.0
ANN_TRN_AMT + AVG_TRN_AMT            0.0
ANN_TRN_AMT + As

In [5]:
# drop the customer_id
df_X = df_raw.drop([config.index_col, config.target_name], axis=1)
df_y = df_raw.loc[:, [config.target_name]]

In [6]:
# train, val, test split
X, X_test, y, y_test = train_test_split(df_X, df_y, test_size=0.1, random_state=1, shuffle=True, stratify=df_y)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=1, shuffle=True, stratify=y)

display(f"train set size: {X_train.shape}, val set size: {X_val.shape}, test set size: {X_test.shape}")

'train set size: (53526, 366), val set size: (5948, 366), test set size: (6609, 366)'

In [8]:
X_train.head()

,C_EDU,C_HSE,INCM_TYP,gn_occ,NUM_PRD,CASATD_CNT,ANN_N_TRX,MTHCASA,MAXCASA,MINCASA,CC_AVE,MAX_MTH_TRN_AMT,MIN_MTH_TRN_AMT,MTHTD,MAXTD,Asset value,AVG_TRN_AMT,ANN_TRN_AMT,CC_LMT,C_AGE,DRvCR,ANN_TRN_AMT + AVG_TRN_AMT,ANN_TRN_AMT + Asset value,ANN_TRN_AMT + CC_AVE,ANN_TRN_AMT + CC_LMT,ANN_TRN_AMT + MAXCASA,ANN_TRN_AMT + MAXTD,ANN_TRN_AMT + MAX_MTH_TRN_AMT,ANN_TRN_AMT + MINCASA,ANN_TRN_AMT + MIN_MTH_TRN_AMT,ANN_TRN_AMT + MTHCASA,ANN_TRN_AMT + MTHTD,AVG_TRN_AMT + CC_AVE,AVG_TRN_AMT + CC_LMT,AVG_TRN_AMT + MAXCASA,AVG_TRN_AMT + MAXTD,AVG_TRN_AMT + MAX_MTH_TRN_AMT,AVG_TRN_AMT + MINCASA,AVG_TRN_AMT + MIN_MTH_TRN_AMT,AVG_TRN_AMT + MTHCASA,AVG_TRN_AMT + MTHTD,Asset value + AVG_TRN_AMT,Asset value + CC_AVE,Asset value + CC_LMT,Asset value + MAXCASA,Asset value + MAXTD,Asset value + MAX_MTH_TRN_AMT,Asset value + MINCASA,Asset value + MIN_MTH_TRN_AMT,Asset value + MTHCASA,Asset value + MTHTD,CC_AVE + CC_LMT,CC_AVE + MAXCASA,CC_AVE + MAXTD,CC_AVE + MAX_MTH_TRN_AMT,CC_AVE + MINCASA,CC_AVE + MIN_MTH_TRN_AMT,CC_AVE + MTHCASA,CC_AVE + MTHTD,CC_LMT + MAXCASA,CC_LMT + MAXTD,CC_LMT + MAX_MTH_TRN_AMT,CC_LMT + MINCASA,CC_LMT + MIN_MTH_TRN_AMT,CC_LMT + MTHCASA,CC_LMT + MTHTD,MAXCASA + MAXTD,MAXCASA + MINCASA,MAXCASA + MIN_MTH_TRN_AMT,MAXCASA + MTHCASA,MAXCASA + MTHTD,MAXTD + MINCASA,MAXTD + MIN_MTH_TRN_AMT,MAXTD + MTHCASA,MAXTD + MTHTD,MAX_MTH_TRN_AMT + MAXCASA,MAX_MTH_TRN_AMT + MAXTD,MAX_MTH_TRN_AMT + MINCASA,MAX_MTH_TRN_AMT + MIN_MTH_TRN_AMT,MAX_MTH_TRN_AMT + MTHCASA,MAX_MTH_TRN_AMT + MTHTD,MINCASA + MTHCASA,MINCASA + MTHTD,MIN_MTH_TRN_AMT + MINCASA,MIN_MTH_TRN_AMT + MTHCASA,MIN_MTH_TRN_AMT + MTHTD,MTHCASA + MTHTD,ANN_TRN_AMT / AVG_TRN_AMT,ANN_TRN_AMT / Asset value,ANN_TRN_AMT / CC_AVE,ANN_TRN_AMT / CC_LMT,ANN_TRN_AMT / MAXCASA,ANN_TRN_AMT / MAXTD,ANN_TRN_AMT / MAX_MTH_TRN_AMT,ANN_TRN_AMT / MINCASA,ANN_TRN_AMT / MIN_MTH_TRN_AMT,ANN_TRN_AMT / MTHCASA,ANN_TRN_AMT / MTHTD,AVG_TRN_AMT / ANN_TRN_AMT,AVG_TRN_AMT / Asset value,AVG_TRN_AMT / CC_AVE,AVG_TRN_AMT / CC_LMT,AVG_TRN_AMT / MAXCASA,AVG_TRN_AMT / MAXTD,AVG_TRN_AMT / MAX_MTH_TRN_AMT,AVG_TRN_AMT / MINCASA,AVG_TRN_AMT / MIN_MTH_TRN_AMT,AVG_TRN_AMT / MTHCASA,AVG_TRN_AMT / MTHTD,Asset value / ANN_TRN_AMT,Asset value / AVG_TRN_AMT,Asset value / CC_AVE,Asset value / CC_LMT,Asset value / MAXCASA,Asset value / MAXTD,Asset value / MAX_MTH_TRN_AMT,Asset value / MINCASA,Asset value / MIN_MTH_TRN_AMT,Asset value / MTHCASA,Asset value / MTHTD,CC_AVE / ANN_TRN_AMT,CC_AVE / AVG_TRN_AMT,CC_AVE / Asset value,CC_AVE / CC_LMT,CC_AVE / MAXCASA,CC_AVE / MAXTD,CC_AVE / MAX_MTH_TRN_AMT,CC_AVE / MINCASA,CC_AVE / MIN_MTH_TRN_AMT,CC_AVE / MTHCASA,CC_AVE / MTHTD,CC_LMT / ANN_TRN_AMT,CC_LMT / AVG_TRN_AMT,CC_LMT / Asset value,CC_LMT / CC_AVE,CC_LMT / MAXCASA,CC_LMT / MAXTD,CC_LMT / MAX_MTH_TRN_AMT,CC_LMT / MINCASA,CC_LMT / MIN_MTH_TRN_AMT,CC_LMT / MTHCASA,CC_LMT / MTHTD,MAXCASA / ANN_TRN_AMT,MAXCASA / AVG_TRN_AMT,MAXCASA / Asset value,MAXCASA / CC_AVE,MAXCASA / CC_LMT,MAXCASA / MAXTD,MAXCASA / MAX_MTH_TRN_AMT,MAXCASA / MINCASA,MAXCASA / MIN_MTH_TRN_AMT,MAXCASA / MTHCASA,MAXCASA / MTHTD,MAXTD / ANN_TRN_AMT,MAXTD / AVG_TRN_AMT,MAXTD / Asset value,MAXTD / CC_AVE,MAXTD / CC_LMT,MAXTD / MAXCASA,MAXTD / MAX_MTH_TRN_AMT,MAXTD / MINCASA,MAXTD / MIN_MTH_TRN_AMT,MAXTD / MTHCASA,MAXTD / MTHTD,MAX_MTH_TRN_AMT / ANN_TRN_AMT,MAX_MTH_TRN_AMT / AVG_TRN_AMT,MAX_MTH_TRN_AMT / Asset value,MAX_MTH_TRN_AMT / CC_AVE,MAX_MTH_TRN_AMT / CC_LMT,MAX_MTH_TRN_AMT / MAXCASA,MAX_MTH_TRN_AMT / MAXTD,MAX_MTH_TRN_AMT / MINCASA,MAX_MTH_TRN_AMT / MIN_MTH_TRN_AMT,MAX_MTH_TRN_AMT / MTHCASA,MAX_MTH_TRN_AMT / MTHTD,MINCASA / ANN_TRN_AMT,MINCASA / AVG_TRN_AMT,MINCASA / Asset value,MINCASA / CC_AVE,MINCASA / CC_LMT,MINCASA / MAXCASA,MINCASA / MAXTD,MINCASA / MAX_MTH_TRN_AMT,MINCASA / MIN_MTH_TRN_AMT,MINCASA / MTHCASA,MINCASA / MTHTD,MIN_MTH_TRN_AMT / ANN_TRN_AMT,MIN_MTH_TRN_AMT / AVG_TRN_AMT,MIN_MTH_TRN_AMT / Asset value,MIN_MTH_TRN_AMT / CC_AVE,MIN_MTH_TRN_AMT / CC_LMT,MIN_MTH_TRN_AMT / MAXCASA,MIN_MTH_TRN_AMT / MAXTD,MIN_MTH_TRN_AMT / MAX_MTH_TRN_AMT,MIN_MTH_TRN_AMT / 

In [10]:
# get cols requiring onehotencoding
categorical = [col for col in (config.ordinal + config.nominal) if col != config.target_name]
categorical.remove('HL_tag')
categorical.remove('AL_tag')

display(f"the categorical variables that need one hot encoding are: {categorical}")

# apply onehotencoding for categorical variables
enc = OneHotEncoder(handle_unknown='error', sparse_output=False, drop=None)
enc.fit_transform(df_X.loc[:, categorical])
feature_labels = enc.get_feature_names_out()


feature_arr = enc.transform(X[categorical])
cat_X = pd.DataFrame(feature_arr, columns=feature_labels).reset_index(drop=True)
X = pd.concat([X.drop(categorical, axis=1).reset_index(drop=True), cat_X], axis=1)

# for training data
feature_arr = enc.transform(X_train[categorical])
cat_train = pd.DataFrame(feature_arr, columns=feature_labels).reset_index(drop=True)
X_train = pd.concat([X_train.drop(categorical, axis=1).reset_index(drop=True), cat_train], axis=1)
display("training data after onehot encoding:", X_train.head())

feature_arr = enc.transform(X_val[categorical])
cat_val = pd.DataFrame(feature_arr, columns=feature_labels).reset_index(drop=True)
X_val = pd.concat([X_val.drop(categorical, axis=1).reset_index(drop=True), cat_val], axis=1)
display("val data after onehot encoding:", X_val.head())

feature_arr = enc.transform(X_test[categorical])
cat_test = pd.DataFrame(feature_arr, columns=feature_labels).reset_index(drop=True)
X_test = pd.concat([X_test.drop(categorical, axis=1).reset_index(drop=True), cat_test], axis=1)
display("test data after onehot encoding:", X_test.head())

"the categorical variables that need one hot encoding are: ['INCM_TYP', 'C_EDU', 'C_HSE', 'gn_occ']"

'training data after onehot encoding:'

,NUM_PRD,CASATD_CNT,ANN_N_TRX,MTHCASA,MAXCASA,MINCASA,CC_AVE,MAX_MTH_TRN_AMT,MIN_MTH_TRN_AMT,MTHTD,MAXTD,Asset value,AVG_TRN_AMT,ANN_TRN_AMT,CC_LMT,C_AGE,DRvCR,ANN_TRN_AMT + AVG_TRN_AMT,ANN_TRN_AMT + Asset value,ANN_TRN_AMT + CC_AVE,ANN_TRN_AMT + CC_LMT,ANN_TRN_AMT + MAXCASA,ANN_TRN_AMT + MAXTD,ANN_TRN_AMT + MAX_MTH_TRN_AMT,ANN_TRN_AMT + MINCASA,ANN_TRN_AMT + MIN_MTH_TRN_AMT,ANN_TRN_AMT + MTHCASA,ANN_TRN_AMT + MTHTD,AVG_TRN_AMT + CC_AVE,AVG_TRN_AMT + CC_LMT,AVG_TRN_AMT + MAXCASA,AVG_TRN_AMT + MAXTD,AVG_TRN_AMT + MAX_MTH_TRN_AMT,AVG_TRN_AMT + MINCASA,AVG_TRN_AMT + MIN_MTH_TRN_AMT,AVG_TRN_AMT + MTHCASA,AVG_TRN_AMT + MTHTD,Asset value + AVG_TRN_AMT,Asset value + CC_AVE,Asset value + CC_LMT,Asset value + MAXCASA,Asset value + MAXTD,Asset value + MAX_MTH_TRN_AMT,Asset value + MINCASA,Asset value + MIN_MTH_TRN_AMT,Asset value + MTHCASA,Asset value + MTHTD,CC_AVE + CC_LMT,CC_AVE + MAXCASA,CC_AVE + MAXTD,CC_AVE + MAX_MTH_TRN_AMT,CC_AVE + MINCASA,CC_AVE + MIN_MTH_TRN_AMT,CC_AVE + MTHCASA,CC_AVE + MTHTD,CC_LMT + MAXCASA,CC_LMT + MAXTD,CC_LMT + MAX_MTH_TRN_AMT,CC_LMT + MINCASA,CC_LMT + MIN_MTH_TRN_AMT,CC_LMT + MTHCASA,CC_LMT + MTHTD,MAXCASA + MAXTD,MAXCASA + MINCASA,MAXCASA + MIN_MTH_TRN_AMT,MAXCASA + MTHCASA,MAXCASA + MTHTD,MAXTD + MINCASA,MAXTD + MIN_MTH_TRN_AMT,MAXTD + MTHCASA,MAXTD + MTHTD,MAX_MTH_TRN_AMT + MAXCASA,MAX_MTH_TRN_AMT + MAXTD,MAX_MTH_TRN_AMT + MINCASA,MAX_MTH_TRN_AMT + MIN_MTH_TRN_AMT,MAX_MTH_TRN_AMT + MTHCASA,MAX_MTH_TRN_AMT + MTHTD,MINCASA + MTHCASA,MINCASA + MTHTD,MIN_MTH_TRN_AMT + MINCASA,MIN_MTH_TRN_AMT + MTHCASA,MIN_MTH_TRN_AMT + MTHTD,MTHCASA + MTHTD,ANN_TRN_AMT / AVG_TRN_AMT,ANN_TRN_AMT / Asset value,ANN_TRN_AMT / CC_AVE,ANN_TRN_AMT / CC_LMT,ANN_TRN_AMT / MAXCASA,ANN_TRN_AMT / MAXTD,ANN_TRN_AMT / MAX_MTH_TRN_AMT,ANN_TRN_AMT / MINCASA,ANN_TRN_AMT / MIN_MTH_TRN_AMT,ANN_TRN_AMT / MTHCASA,ANN_TRN_AMT / MTHTD,AVG_TRN_AMT / ANN_TRN_AMT,AVG_TRN_AMT / Asset value,AVG_TRN_AMT / CC_AVE,AVG_TRN_AMT / CC_LMT,AVG_TRN_AMT / MAXCASA,AVG_TRN_AMT / MAXTD,AVG_TRN_AMT / MAX_MTH_TRN_AMT,AVG_TRN_AMT / MINCASA,AVG_TRN_AMT / MIN_MTH_TRN_AMT,AVG_TRN_AMT / MTHCASA,AVG_TRN_AMT / MTHTD,Asset value / ANN_TRN_AMT,Asset value / AVG_TRN_AMT,Asset value / CC_AVE,Asset value / CC_LMT,Asset value / MAXCASA,Asset value / MAXTD,Asset value / MAX_MTH_TRN_AMT,Asset value / MINCASA,Asset value / MIN_MTH_TRN_AMT,Asset value / MTHCASA,Asset value / MTHTD,CC_AVE / ANN_TRN_AMT,CC_AVE / AVG_TRN_AMT,CC_AVE / Asset value,CC_AVE / CC_LMT,CC_AVE / MAXCASA,CC_AVE / MAXTD,CC_AVE / MAX_MTH_TRN_AMT,CC_AVE / MINCASA,CC_AVE / MIN_MTH_TRN_AMT,CC_AVE / MTHCASA,CC_AVE / MTHTD,CC_LMT / ANN_TRN_AMT,CC_LMT / AVG_TRN_AMT,CC_LMT / Asset value,CC_LMT / CC_AVE,CC_LMT / MAXCASA,CC_LMT / MAXTD,CC_LMT / MAX_MTH_TRN_AMT,CC_LMT / MINCASA,CC_LMT / MIN_MTH_TRN_AMT,CC_LMT / MTHCASA,CC_LMT / MTHTD,MAXCASA / ANN_TRN_AMT,MAXCASA / AVG_TRN_AMT,MAXCASA / Asset value,MAXCASA / CC_AVE,MAXCASA / CC_LMT,MAXCASA / MAXTD,MAXCASA / MAX_MTH_TRN_AMT,MAXCASA / MINCASA,MAXCASA / MIN_MTH_TRN_AMT,MAXCASA / MTHCASA,MAXCASA / MTHTD,MAXTD / ANN_TRN_AMT,MAXTD / AVG_TRN_AMT,MAXTD / Asset value,MAXTD / CC_AVE,MAXTD / CC_LMT,MAXTD / MAXCASA,MAXTD / MAX_MTH_TRN_AMT,MAXTD / MINCASA,MAXTD / MIN_MTH_TRN_AMT,MAXTD / MTHCASA,MAXTD / MTHTD,MAX_MTH_TRN_AMT / ANN_TRN_AMT,MAX_MTH_TRN_AMT / AVG_TRN_AMT,MAX_MTH_TRN_AMT / Asset value,MAX_MTH_TRN_AMT / CC_AVE,MAX_MTH_TRN_AMT / CC_LMT,MAX_MTH_TRN_AMT / MAXCASA,MAX_MTH_TRN_AMT / MAXTD,MAX_MTH_TRN_AMT / MINCASA,MAX_MTH_TRN_AMT / MIN_MTH_TRN_AMT,MAX_MTH_TRN_AMT / MTHCASA,MAX_MTH_TRN_AMT / MTHTD,MINCASA / ANN_TRN_AMT,MINCASA / AVG_TRN_AMT,MINCASA / Asset value,MINCASA / CC_AVE,MINCASA / CC_LMT,MINCASA / MAXCASA,MINCASA / MAXTD,MINCASA / MAX_MTH_TRN_AMT,MINCASA / MIN_MTH_TRN_AMT,MINCASA / MTHCASA,MINCASA / MTHTD,MIN_MTH_TRN_AMT / ANN_TRN_AMT,MIN_MTH_TRN_AMT / AVG_TRN_AMT,MIN_MTH_TRN_AMT / Asset value,MIN_MTH_TRN_AMT / CC_AVE,MIN_MTH_TRN_AMT / CC_LMT,MIN_MTH_TRN_AMT / MAXCASA,MIN_MTH_TRN_AMT / MAXTD,MIN_MTH_TRN_AMT / MAX_MTH_TRN_AMT,MIN_MTH_TRN_AMT / MINCASA,MIN_MTH_TRN_AMT / MT

'val data after onehot encoding:'

,NUM_PRD,CASATD_CNT,ANN_N_TRX,MTHCASA,MAXCASA,MINCASA,CC_AVE,MAX_MTH_TRN_AMT,MIN_MTH_TRN_AMT,MTHTD,MAXTD,Asset value,AVG_TRN_AMT,ANN_TRN_AMT,CC_LMT,C_AGE,DRvCR,ANN_TRN_AMT + AVG_TRN_AMT,ANN_TRN_AMT + Asset value,ANN_TRN_AMT + CC_AVE,ANN_TRN_AMT + CC_LMT,ANN_TRN_AMT + MAXCASA,ANN_TRN_AMT + MAXTD,ANN_TRN_AMT + MAX_MTH_TRN_AMT,ANN_TRN_AMT + MINCASA,ANN_TRN_AMT + MIN_MTH_TRN_AMT,ANN_TRN_AMT + MTHCASA,ANN_TRN_AMT + MTHTD,AVG_TRN_AMT + CC_AVE,AVG_TRN_AMT + CC_LMT,AVG_TRN_AMT + MAXCASA,AVG_TRN_AMT + MAXTD,AVG_TRN_AMT + MAX_MTH_TRN_AMT,AVG_TRN_AMT + MINCASA,AVG_TRN_AMT + MIN_MTH_TRN_AMT,AVG_TRN_AMT + MTHCASA,AVG_TRN_AMT + MTHTD,Asset value + AVG_TRN_AMT,Asset value + CC_AVE,Asset value + CC_LMT,Asset value + MAXCASA,Asset value + MAXTD,Asset value + MAX_MTH_TRN_AMT,Asset value + MINCASA,Asset value + MIN_MTH_TRN_AMT,Asset value + MTHCASA,Asset value + MTHTD,CC_AVE + CC_LMT,CC_AVE + MAXCASA,CC_AVE + MAXTD,CC_AVE + MAX_MTH_TRN_AMT,CC_AVE + MINCASA,CC_AVE + MIN_MTH_TRN_AMT,CC_AVE + MTHCASA,CC_AVE + MTHTD,CC_LMT + MAXCASA,CC_LMT + MAXTD,CC_LMT + MAX_MTH_TRN_AMT,CC_LMT + MINCASA,CC_LMT + MIN_MTH_TRN_AMT,CC_LMT + MTHCASA,CC_LMT + MTHTD,MAXCASA + MAXTD,MAXCASA + MINCASA,MAXCASA + MIN_MTH_TRN_AMT,MAXCASA + MTHCASA,MAXCASA + MTHTD,MAXTD + MINCASA,MAXTD + MIN_MTH_TRN_AMT,MAXTD + MTHCASA,MAXTD + MTHTD,MAX_MTH_TRN_AMT + MAXCASA,MAX_MTH_TRN_AMT + MAXTD,MAX_MTH_TRN_AMT + MINCASA,MAX_MTH_TRN_AMT + MIN_MTH_TRN_AMT,MAX_MTH_TRN_AMT + MTHCASA,MAX_MTH_TRN_AMT + MTHTD,MINCASA + MTHCASA,MINCASA + MTHTD,MIN_MTH_TRN_AMT + MINCASA,MIN_MTH_TRN_AMT + MTHCASA,MIN_MTH_TRN_AMT + MTHTD,MTHCASA + MTHTD,ANN_TRN_AMT / AVG_TRN_AMT,ANN_TRN_AMT / Asset value,ANN_TRN_AMT / CC_AVE,ANN_TRN_AMT / CC_LMT,ANN_TRN_AMT / MAXCASA,ANN_TRN_AMT / MAXTD,ANN_TRN_AMT / MAX_MTH_TRN_AMT,ANN_TRN_AMT / MINCASA,ANN_TRN_AMT / MIN_MTH_TRN_AMT,ANN_TRN_AMT / MTHCASA,ANN_TRN_AMT / MTHTD,AVG_TRN_AMT / ANN_TRN_AMT,AVG_TRN_AMT / Asset value,AVG_TRN_AMT / CC_AVE,AVG_TRN_AMT / CC_LMT,AVG_TRN_AMT / MAXCASA,AVG_TRN_AMT / MAXTD,AVG_TRN_AMT / MAX_MTH_TRN_AMT,AVG_TRN_AMT / MINCASA,AVG_TRN_AMT / MIN_MTH_TRN_AMT,AVG_TRN_AMT / MTHCASA,AVG_TRN_AMT / MTHTD,Asset value / ANN_TRN_AMT,Asset value / AVG_TRN_AMT,Asset value / CC_AVE,Asset value / CC_LMT,Asset value / MAXCASA,Asset value / MAXTD,Asset value / MAX_MTH_TRN_AMT,Asset value / MINCASA,Asset value / MIN_MTH_TRN_AMT,Asset value / MTHCASA,Asset value / MTHTD,CC_AVE / ANN_TRN_AMT,CC_AVE / AVG_TRN_AMT,CC_AVE / Asset value,CC_AVE / CC_LMT,CC_AVE / MAXCASA,CC_AVE / MAXTD,CC_AVE / MAX_MTH_TRN_AMT,CC_AVE / MINCASA,CC_AVE / MIN_MTH_TRN_AMT,CC_AVE / MTHCASA,CC_AVE / MTHTD,CC_LMT / ANN_TRN_AMT,CC_LMT / AVG_TRN_AMT,CC_LMT / Asset value,CC_LMT / CC_AVE,CC_LMT / MAXCASA,CC_LMT / MAXTD,CC_LMT / MAX_MTH_TRN_AMT,CC_LMT / MINCASA,CC_LMT / MIN_MTH_TRN_AMT,CC_LMT / MTHCASA,CC_LMT / MTHTD,MAXCASA / ANN_TRN_AMT,MAXCASA / AVG_TRN_AMT,MAXCASA / Asset value,MAXCASA / CC_AVE,MAXCASA / CC_LMT,MAXCASA / MAXTD,MAXCASA / MAX_MTH_TRN_AMT,MAXCASA / MINCASA,MAXCASA / MIN_MTH_TRN_AMT,MAXCASA / MTHCASA,MAXCASA / MTHTD,MAXTD / ANN_TRN_AMT,MAXTD / AVG_TRN_AMT,MAXTD / Asset value,MAXTD / CC_AVE,MAXTD / CC_LMT,MAXTD / MAXCASA,MAXTD / MAX_MTH_TRN_AMT,MAXTD / MINCASA,MAXTD / MIN_MTH_TRN_AMT,MAXTD / MTHCASA,MAXTD / MTHTD,MAX_MTH_TRN_AMT / ANN_TRN_AMT,MAX_MTH_TRN_AMT / AVG_TRN_AMT,MAX_MTH_TRN_AMT / Asset value,MAX_MTH_TRN_AMT / CC_AVE,MAX_MTH_TRN_AMT / CC_LMT,MAX_MTH_TRN_AMT / MAXCASA,MAX_MTH_TRN_AMT / MAXTD,MAX_MTH_TRN_AMT / MINCASA,MAX_MTH_TRN_AMT / MIN_MTH_TRN_AMT,MAX_MTH_TRN_AMT / MTHCASA,MAX_MTH_TRN_AMT / MTHTD,MINCASA / ANN_TRN_AMT,MINCASA / AVG_TRN_AMT,MINCASA / Asset value,MINCASA / CC_AVE,MINCASA / CC_LMT,MINCASA / MAXCASA,MINCASA / MAXTD,MINCASA / MAX_MTH_TRN_AMT,MINCASA / MIN_MTH_TRN_AMT,MINCASA / MTHCASA,MINCASA / MTHTD,MIN_MTH_TRN_AMT / ANN_TRN_AMT,MIN_MTH_TRN_AMT / AVG_TRN_AMT,MIN_MTH_TRN_AMT / Asset value,MIN_MTH_TRN_AMT / CC_AVE,MIN_MTH_TRN_AMT / CC_LMT,MIN_MTH_TRN_AMT / MAXCASA,MIN_MTH_TRN_AMT / MAXTD,MIN_MTH_TRN_AMT / MAX_MTH_TRN_AMT,MIN_MTH_TRN_AMT / MINCASA,MIN_MTH_TRN_AMT / MT

'test data after onehot encoding:'

,NUM_PRD,CASATD_CNT,ANN_N_TRX,MTHCASA,MAXCASA,MINCASA,CC_AVE,MAX_MTH_TRN_AMT,MIN_MTH_TRN_AMT,MTHTD,MAXTD,Asset value,AVG_TRN_AMT,ANN_TRN_AMT,CC_LMT,C_AGE,DRvCR,ANN_TRN_AMT + AVG_TRN_AMT,ANN_TRN_AMT + Asset value,ANN_TRN_AMT + CC_AVE,ANN_TRN_AMT + CC_LMT,ANN_TRN_AMT + MAXCASA,ANN_TRN_AMT + MAXTD,ANN_TRN_AMT + MAX_MTH_TRN_AMT,ANN_TRN_AMT + MINCASA,ANN_TRN_AMT + MIN_MTH_TRN_AMT,ANN_TRN_AMT + MTHCASA,ANN_TRN_AMT + MTHTD,AVG_TRN_AMT + CC_AVE,AVG_TRN_AMT + CC_LMT,AVG_TRN_AMT + MAXCASA,AVG_TRN_AMT + MAXTD,AVG_TRN_AMT + MAX_MTH_TRN_AMT,AVG_TRN_AMT + MINCASA,AVG_TRN_AMT + MIN_MTH_TRN_AMT,AVG_TRN_AMT + MTHCASA,AVG_TRN_AMT + MTHTD,Asset value + AVG_TRN_AMT,Asset value + CC_AVE,Asset value + CC_LMT,Asset value + MAXCASA,Asset value + MAXTD,Asset value + MAX_MTH_TRN_AMT,Asset value + MINCASA,Asset value + MIN_MTH_TRN_AMT,Asset value + MTHCASA,Asset value + MTHTD,CC_AVE + CC_LMT,CC_AVE + MAXCASA,CC_AVE + MAXTD,CC_AVE + MAX_MTH_TRN_AMT,CC_AVE + MINCASA,CC_AVE + MIN_MTH_TRN_AMT,CC_AVE + MTHCASA,CC_AVE + MTHTD,CC_LMT + MAXCASA,CC_LMT + MAXTD,CC_LMT + MAX_MTH_TRN_AMT,CC_LMT + MINCASA,CC_LMT + MIN_MTH_TRN_AMT,CC_LMT + MTHCASA,CC_LMT + MTHTD,MAXCASA + MAXTD,MAXCASA + MINCASA,MAXCASA + MIN_MTH_TRN_AMT,MAXCASA + MTHCASA,MAXCASA + MTHTD,MAXTD + MINCASA,MAXTD + MIN_MTH_TRN_AMT,MAXTD + MTHCASA,MAXTD + MTHTD,MAX_MTH_TRN_AMT + MAXCASA,MAX_MTH_TRN_AMT + MAXTD,MAX_MTH_TRN_AMT + MINCASA,MAX_MTH_TRN_AMT + MIN_MTH_TRN_AMT,MAX_MTH_TRN_AMT + MTHCASA,MAX_MTH_TRN_AMT + MTHTD,MINCASA + MTHCASA,MINCASA + MTHTD,MIN_MTH_TRN_AMT + MINCASA,MIN_MTH_TRN_AMT + MTHCASA,MIN_MTH_TRN_AMT + MTHTD,MTHCASA + MTHTD,ANN_TRN_AMT / AVG_TRN_AMT,ANN_TRN_AMT / Asset value,ANN_TRN_AMT / CC_AVE,ANN_TRN_AMT / CC_LMT,ANN_TRN_AMT / MAXCASA,ANN_TRN_AMT / MAXTD,ANN_TRN_AMT / MAX_MTH_TRN_AMT,ANN_TRN_AMT / MINCASA,ANN_TRN_AMT / MIN_MTH_TRN_AMT,ANN_TRN_AMT / MTHCASA,ANN_TRN_AMT / MTHTD,AVG_TRN_AMT / ANN_TRN_AMT,AVG_TRN_AMT / Asset value,AVG_TRN_AMT / CC_AVE,AVG_TRN_AMT / CC_LMT,AVG_TRN_AMT / MAXCASA,AVG_TRN_AMT / MAXTD,AVG_TRN_AMT / MAX_MTH_TRN_AMT,AVG_TRN_AMT / MINCASA,AVG_TRN_AMT / MIN_MTH_TRN_AMT,AVG_TRN_AMT / MTHCASA,AVG_TRN_AMT / MTHTD,Asset value / ANN_TRN_AMT,Asset value / AVG_TRN_AMT,Asset value / CC_AVE,Asset value / CC_LMT,Asset value / MAXCASA,Asset value / MAXTD,Asset value / MAX_MTH_TRN_AMT,Asset value / MINCASA,Asset value / MIN_MTH_TRN_AMT,Asset value / MTHCASA,Asset value / MTHTD,CC_AVE / ANN_TRN_AMT,CC_AVE / AVG_TRN_AMT,CC_AVE / Asset value,CC_AVE / CC_LMT,CC_AVE / MAXCASA,CC_AVE / MAXTD,CC_AVE / MAX_MTH_TRN_AMT,CC_AVE / MINCASA,CC_AVE / MIN_MTH_TRN_AMT,CC_AVE / MTHCASA,CC_AVE / MTHTD,CC_LMT / ANN_TRN_AMT,CC_LMT / AVG_TRN_AMT,CC_LMT / Asset value,CC_LMT / CC_AVE,CC_LMT / MAXCASA,CC_LMT / MAXTD,CC_LMT / MAX_MTH_TRN_AMT,CC_LMT / MINCASA,CC_LMT / MIN_MTH_TRN_AMT,CC_LMT / MTHCASA,CC_LMT / MTHTD,MAXCASA / ANN_TRN_AMT,MAXCASA / AVG_TRN_AMT,MAXCASA / Asset value,MAXCASA / CC_AVE,MAXCASA / CC_LMT,MAXCASA / MAXTD,MAXCASA / MAX_MTH_TRN_AMT,MAXCASA / MINCASA,MAXCASA / MIN_MTH_TRN_AMT,MAXCASA / MTHCASA,MAXCASA / MTHTD,MAXTD / ANN_TRN_AMT,MAXTD / AVG_TRN_AMT,MAXTD / Asset value,MAXTD / CC_AVE,MAXTD / CC_LMT,MAXTD / MAXCASA,MAXTD / MAX_MTH_TRN_AMT,MAXTD / MINCASA,MAXTD / MIN_MTH_TRN_AMT,MAXTD / MTHCASA,MAXTD / MTHTD,MAX_MTH_TRN_AMT / ANN_TRN_AMT,MAX_MTH_TRN_AMT / AVG_TRN_AMT,MAX_MTH_TRN_AMT / Asset value,MAX_MTH_TRN_AMT / CC_AVE,MAX_MTH_TRN_AMT / CC_LMT,MAX_MTH_TRN_AMT / MAXCASA,MAX_MTH_TRN_AMT / MAXTD,MAX_MTH_TRN_AMT / MINCASA,MAX_MTH_TRN_AMT / MIN_MTH_TRN_AMT,MAX_MTH_TRN_AMT / MTHCASA,MAX_MTH_TRN_AMT / MTHTD,MINCASA / ANN_TRN_AMT,MINCASA / AVG_TRN_AMT,MINCASA / Asset value,MINCASA / CC_AVE,MINCASA / CC_LMT,MINCASA / MAXCASA,MINCASA / MAXTD,MINCASA / MAX_MTH_TRN_AMT,MINCASA / MIN_MTH_TRN_AMT,MINCASA / MTHCASA,MINCASA / MTHTD,MIN_MTH_TRN_AMT / ANN_TRN_AMT,MIN_MTH_TRN_AMT / AVG_TRN_AMT,MIN_MTH_TRN_AMT / Asset value,MIN_MTH_TRN_AMT / CC_AVE,MIN_MTH_TRN_AMT / CC_LMT,MIN_MTH_TRN_AMT / MAXCASA,MIN_MTH_TRN_AMT / MAXTD,MIN_MTH_TRN_AMT / MAX_MTH_TRN_AMT,MIN_MTH_TRN_AMT / MINCASA,MIN_MTH_TRN_AMT / MT

In [11]:
# # try to drop useless columns based on feature selection
useless = ['C_HSE_OFFICE',
 'ANN_TRN_AMT / AVG_TRN_AMT',
 'C_HSE_COMMERICAL BUILDING',
 'AVG_TRN_AMT / ANN_TRN_AMT',
 'C_HSE_INDUSTRIAL BUILDING',
 'C_HSE_HOTEL/ SERVICE APARTMENT']

X.drop(useless, axis = 1, inplace=True)
X_train.drop(useless, axis = 1, inplace=True)
X_val.drop(useless, axis = 1, inplace=True)
X_test.drop(useless, axis = 1, inplace=True)

In [12]:
# label encoding for target variable
le = preprocessing.LabelEncoder()

y = pd.DataFrame(le.fit_transform(y[config.target_name]), columns=["Y"])

y_train = pd.DataFrame(le.transform(y_train[config.target_name]), columns=["Y"])
display("training after label encoding:", y_train.head())

y_val = pd.DataFrame(le.transform(y_val[config.target_name]), columns=["Y"])
display("y_val after label encoding:", y_val.head())

y_test = pd.DataFrame(le.transform(y_test[config.target_name]), columns=["Y"])
display("y_test after label encoding:", y_test.head())

'training after label encoding:'

,Y
0,1
1,1
2,1
3,1
4,0


'y_val after label encoding:'

,Y
0,1
1,1
2,1
3,0
4,1


'y_test after label encoding:'

,Y
0,1
1,1
2,1
3,0
4,1


In [13]:
# upsamping
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=1, k_neighbors=5)

X_train, y_train = sm.fit_resample(X_train, y_train)

In [14]:
X_train.head()

,NUM_PRD,CASATD_CNT,ANN_N_TRX,MTHCASA,MAXCASA,MINCASA,CC_AVE,MAX_MTH_TRN_AMT,MIN_MTH_TRN_AMT,MTHTD,MAXTD,Asset value,AVG_TRN_AMT,ANN_TRN_AMT,CC_LMT,C_AGE,DRvCR,ANN_TRN_AMT + AVG_TRN_AMT,ANN_TRN_AMT + Asset value,ANN_TRN_AMT + CC_AVE,ANN_TRN_AMT + CC_LMT,ANN_TRN_AMT + MAXCASA,ANN_TRN_AMT + MAXTD,ANN_TRN_AMT + MAX_MTH_TRN_AMT,ANN_TRN_AMT + MINCASA,ANN_TRN_AMT + MIN_MTH_TRN_AMT,ANN_TRN_AMT + MTHCASA,ANN_TRN_AMT + MTHTD,AVG_TRN_AMT + CC_AVE,AVG_TRN_AMT + CC_LMT,AVG_TRN_AMT + MAXCASA,AVG_TRN_AMT + MAXTD,AVG_TRN_AMT + MAX_MTH_TRN_AMT,AVG_TRN_AMT + MINCASA,AVG_TRN_AMT + MIN_MTH_TRN_AMT,AVG_TRN_AMT + MTHCASA,AVG_TRN_AMT + MTHTD,Asset value + AVG_TRN_AMT,Asset value + CC_AVE,Asset value + CC_LMT,Asset value + MAXCASA,Asset value + MAXTD,Asset value + MAX_MTH_TRN_AMT,Asset value + MINCASA,Asset value + MIN_MTH_TRN_AMT,Asset value + MTHCASA,Asset value + MTHTD,CC_AVE + CC_LMT,CC_AVE + MAXCASA,CC_AVE + MAXTD,CC_AVE + MAX_MTH_TRN_AMT,CC_AVE + MINCASA,CC_AVE + MIN_MTH_TRN_AMT,CC_AVE + MTHCASA,CC_AVE + MTHTD,CC_LMT + MAXCASA,CC_LMT + MAXTD,CC_LMT + MAX_MTH_TRN_AMT,CC_LMT + MINCASA,CC_LMT + MIN_MTH_TRN_AMT,CC_LMT + MTHCASA,CC_LMT + MTHTD,MAXCASA + MAXTD,MAXCASA + MINCASA,MAXCASA + MIN_MTH_TRN_AMT,MAXCASA + MTHCASA,MAXCASA + MTHTD,MAXTD + MINCASA,MAXTD + MIN_MTH_TRN_AMT,MAXTD + MTHCASA,MAXTD + MTHTD,MAX_MTH_TRN_AMT + MAXCASA,MAX_MTH_TRN_AMT + MAXTD,MAX_MTH_TRN_AMT + MINCASA,MAX_MTH_TRN_AMT + MIN_MTH_TRN_AMT,MAX_MTH_TRN_AMT + MTHCASA,MAX_MTH_TRN_AMT + MTHTD,MINCASA + MTHCASA,MINCASA + MTHTD,MIN_MTH_TRN_AMT + MINCASA,MIN_MTH_TRN_AMT + MTHCASA,MIN_MTH_TRN_AMT + MTHTD,MTHCASA + MTHTD,ANN_TRN_AMT / Asset value,ANN_TRN_AMT / CC_AVE,ANN_TRN_AMT / CC_LMT,ANN_TRN_AMT / MAXCASA,ANN_TRN_AMT / MAXTD,ANN_TRN_AMT / MAX_MTH_TRN_AMT,ANN_TRN_AMT / MINCASA,ANN_TRN_AMT / MIN_MTH_TRN_AMT,ANN_TRN_AMT / MTHCASA,ANN_TRN_AMT / MTHTD,AVG_TRN_AMT / Asset value,AVG_TRN_AMT / CC_AVE,AVG_TRN_AMT / CC_LMT,AVG_TRN_AMT / MAXCASA,AVG_TRN_AMT / MAXTD,AVG_TRN_AMT / MAX_MTH_TRN_AMT,AVG_TRN_AMT / MINCASA,AVG_TRN_AMT / MIN_MTH_TRN_AMT,AVG_TRN_AMT / MTHCASA,AVG_TRN_AMT / MTHTD,Asset value / ANN_TRN_AMT,Asset value / AVG_TRN_AMT,Asset value / CC_AVE,Asset value / CC_LMT,Asset value / MAXCASA,Asset value / MAXTD,Asset value / MAX_MTH_TRN_AMT,Asset value / MINCASA,Asset value / MIN_MTH_TRN_AMT,Asset value / MTHCASA,Asset value / MTHTD,CC_AVE / ANN_TRN_AMT,CC_AVE / AVG_TRN_AMT,CC_AVE / Asset value,CC_AVE / CC_LMT,CC_AVE / MAXCASA,CC_AVE / MAXTD,CC_AVE / MAX_MTH_TRN_AMT,CC_AVE / MINCASA,CC_AVE / MIN_MTH_TRN_AMT,CC_AVE / MTHCASA,CC_AVE / MTHTD,CC_LMT / ANN_TRN_AMT,CC_LMT / AVG_TRN_AMT,CC_LMT / Asset value,CC_LMT / CC_AVE,CC_LMT / MAXCASA,CC_LMT / MAXTD,CC_LMT / MAX_MTH_TRN_AMT,CC_LMT / MINCASA,CC_LMT / MIN_MTH_TRN_AMT,CC_LMT / MTHCASA,CC_LMT / MTHTD,MAXCASA / ANN_TRN_AMT,MAXCASA / AVG_TRN_AMT,MAXCASA / Asset value,MAXCASA / CC_AVE,MAXCASA / CC_LMT,MAXCASA / MAXTD,MAXCASA / MAX_MTH_TRN_AMT,MAXCASA / MINCASA,MAXCASA / MIN_MTH_TRN_AMT,MAXCASA / MTHCASA,MAXCASA / MTHTD,MAXTD / ANN_TRN_AMT,MAXTD / AVG_TRN_AMT,MAXTD / Asset value,MAXTD / CC_AVE,MAXTD / CC_LMT,MAXTD / MAXCASA,MAXTD / MAX_MTH_TRN_AMT,MAXTD / MINCASA,MAXTD / MIN_MTH_TRN_AMT,MAXTD / MTHCASA,MAXTD / MTHTD,MAX_MTH_TRN_AMT / ANN_TRN_AMT,MAX_MTH_TRN_AMT / AVG_TRN_AMT,MAX_MTH_TRN_AMT / Asset value,MAX_MTH_TRN_AMT / CC_AVE,MAX_MTH_TRN_AMT / CC_LMT,MAX_MTH_TRN_AMT / MAXCASA,MAX_MTH_TRN_AMT / MAXTD,MAX_MTH_TRN_AMT / MINCASA,MAX_MTH_TRN_AMT / MIN_MTH_TRN_AMT,MAX_MTH_TRN_AMT / MTHCASA,MAX_MTH_TRN_AMT / MTHTD,MINCASA / ANN_TRN_AMT,MINCASA / AVG_TRN_AMT,MINCASA / Asset value,MINCASA / CC_AVE,MINCASA / CC_LMT,MINCASA / MAXCASA,MINCASA / MAXTD,MINCASA / MAX_MTH_TRN_AMT,MINCASA / MIN_MTH_TRN_AMT,MINCASA / MTHCASA,MINCASA / MTHTD,MIN_MTH_TRN_AMT / ANN_TRN_AMT,MIN_MTH_TRN_AMT / AVG_TRN_AMT,MIN_MTH_TRN_AMT / Asset value,MIN_MTH_TRN_AMT / CC_AVE,MIN_MTH_TRN_AMT / CC_LMT,MIN_MTH_TRN_AMT / MAXCASA,MIN_MTH_TRN_AMT / MAXTD,MIN_MTH_TRN_AMT / MAX_MTH_TRN_AMT,MIN_MTH_TRN_AMT / MINCASA,MIN_MTH_TRN_AMT / MTHCASA,MIN_MTH_TRN_AMT / MTHTD,MTHCASA / ANN_TRN_AMT,

In [15]:
clf = MLPClassifier(random_state=1, max_iter=100).fit(X_train, y_train)

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


array([[0.04080792, 0.95919208]])

In [17]:
clf.predict(X_test[:1])

array([1])

In [19]:
# predict on test set
preds = clf.predict(X_test)

# 0.9228875406664908
f1_score(y_test, preds)

0.8854531310774019